# Реализуем метод predict_proba для KNN

Ниже реализован класс KNeighborsClassifier, который для поиска ближайших соседей использует sklearn.neighbors.NearestNeighbors

Требуется реализовать метод predict_proba для вычисления ответа классификатора.

In [1]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors


class KNeighborsClassifier(BaseEstimator, ClassifierMixin):
    '''
    Класс, который позволит нам изучить KNN
    '''
    def __init__(self, n_neighbors=5, weights='uniform', 
                 metric='minkowski', p=2):
        '''
        Инициализируем KNN с несколькими стандартными параметрами
        '''
        assert weights in ('uniform', 'distance')
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        
        self.NearestNeighbors = NearestNeighbors(
            n_neighbors = n_neighbors,
            metric = self.metric)
    
    def fit(self, X, y):
        '''
        Используем sklearn.neighbors.NearestNeighbors 
        для запоминания обучающей выборки
        и последующего поиска соседей
        '''
        self.NearestNeighbors.fit(X)
        self.n_classes = len(np.unique(y))
        self.y = y
        
    def predict_proba(self, X, use_first_zero_distant_sample=True):
        '''
        Чтобы реализовать этот метод, 
        изучите работу sklearn.neighbors.NearestNeighbors'''
        
        distances, inds = self.NearestNeighbors.kneighbors(X)
        labels = np.array([self.y[i] for i in inds])
        
        # получим здесь расстояния до соседей distances и их метки
        
        if self.weights == 'uniform':
            w = np.ones(distances.shape)
        else:
            # чтобы не делить на 0, 
            # добавим небольшую константу, например 1e-3
            w = 1/(distances + 1e-3)

        classes = np.unique(self.y)

        probs = np.empty(shape = (len(X), len(classes)), dtype=np.ndarray)
        for obj_index in range(len(X)):
            nearest_neighbors = zip(w[obj_index], labels[obj_index])
            probs_map = {c:0 for c in classes}
            weight_sum = 0
            for distance, label in nearest_neighbors:
                probs_map[label] += distance;
                weight_sum += distance;
            probs[obj_index] = np.array([probs_map[l] / weight_sum for l in classes])
            #print(obj_index, w[obj_index], labels[obj_index], probs[obj_index])
            
        # реализуем вычисление предсказаний:
        # выбрав один объект, для каждого класса посчитаем
        # суммарный вес голосующих за него объектов
        # затем нормируем эти веса на их сумму
        # и вернем это как предсказание KNN
        return probs

# Загрузим данные и обучим классификатор

In [2]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(weights='distance')
knn.fit(X, y)
prediction = knn.predict_proba(X, )

Поскольку мы используем одну и ту же выборку для обучения и предсказания, ближайшим соседом любого объекта будет он же сам. В качестве упражнения предлагаю реализовать метод transform, который реализует получение предсказаний для обучающей выборки, но для каждого объекта не будет учитывать его самого.

Посмотрим, в каких объектах max(prediction) != 1:

In [3]:
inds = np.arange(len(prediction))[prediction.max(1) != 1]
print(inds)

# [ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]

[ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]


Несколько примеров, на которых можно проверить правильность реализованного метода:

In [4]:
for i in 1, 4, -1:
    print(inds[i], prediction[inds[i]])

# 68 [0.         0.99816311 0.00183689]
# 77 [0.         0.99527902 0.00472098]
# 146 [0.         0.00239145 0.99760855]

68 [0.0 0.998163109597907 0.001836890402092989]
77 [0.0 0.9952790210991134 0.0047209789008865475]
146 [0.0 0.0023914455604554055 0.9976085544395445]


# Ответы для формы

В форму требуется ввести max(prediction) для объекта. Если метод реализован верно, то ячейка ниже распечатает ответы, которые нужно ввести в форму

In [5]:
for i in 56, 83, 127:
    print('{:.4f}'.format(max(prediction[i])))

0.9978
0.9889
0.9967


In [6]:
samples = [[5.1, 3.5, 1.4, 0.2],
[4.9, 3.,  1.4, 0.2],
[4.7, 3.2, 1.3, 0.2],
[4.6, 3.1, 1.5, 0.2]]
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=2)
neigh.fit(samples) 
NearestNeighbors(algorithm='auto', metric='l1', leaf_size=30)
print(neigh.kneighbors([[4.9, 3.,  1.4, 0.2]])) 


(array([[0. , 0.3]]), array([[1, 2]]))


In [7]:
x = [1,2,3]
y = {r:0 for r in x}
print(y)

{1: 0, 2: 0, 3: 0}


In [8]:
x = np.array([1.,2.,3.])
y = np.empty(shape = (2,3), dtype=np.ndarray)


In [9]:
y [0] = x

In [10]:
print(y)

[[1.0 2.0 3.0]
 [None None None]]
